In [1]:
import scirpy as ir
import muon as mu
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

# Query epitope databases
- search for TCRs with specificity annotatation from previous studies.

**Databases**
- [VDJdb](https://vdjdb.cdr3.net/), a curated database of T-cell receptor (TCR) sequences with known antigen specificities.
- [IEDB](https://www.iedb.org/): immune epitope data related to all species studied and includes antibody, T cell, and MHC binding contexts associated with infectious, allergic, autoimmune, and transplant related diseases. IEDB [Fleri et al., 2017]
- [McPAS-TCR](http://friedmanlab.weizmann.ac.il/McPAS-TCR/)
- [PIRD](https://db.cngb.org/pird/)
- [immuneCodeTM](https://clients.adaptivebiotech.com/pub/covid-2020) for covid specific TCRs

In [2]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    mdata = mu.read_h5mu("../data/tcr_data/121c_clone_id_full.h5mu")

    ### Split in B- and T- cells
    b_cells = {'B intermediate', 'Plasmablast'}
    t_cells = {'gdT', 'CD4 Memory', 'CD8 Memory', 'CD4 Naive', 'CD8 Proliferating', 'CD8 Naive', 'CD4 Proliferating', 'Treg'}
    
    bcell_mdata = mdata[mdata["gex"].obs["consensus_celltype"].isin(b_cells)].copy()
    tcell_mdata = mdata[mdata["gex"].obs["consensus_celltype"].isin(t_cells)].copy()

In [3]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    iedb = ir.datasets.iedb()
    vdjdb = ir.datasets.vdjdb()

### Look at the HLA molecules.

In [4]:
vdjdb.obs.value_counts("mhc.a").iloc[:10]

mhc.a
HLA-A*02       14880
HLA-A*03:01    14714
HLA-A*02:01    10521
HLA-A*11:01     2942
Mamu-A*01       2119
HLA-B*07:02     1981
HLA-B*08:01     1693
H-2Kb           1691
HLA-A*01:01     1583
HLA-DRA*01      1414
Name: count, dtype: int64

In [5]:
print(vdjdb.obs.value_counts("antigen.species").iloc[:10])
print("*" * 50)
print(f"There are {len(vdjdb.obs.value_counts('antigen.species'))} antigen species")

antigen.species
CMV            23590
InfluenzaA     11194
EBV             6661
SARS-CoV-2      6237
HomoSapiens     4131
HIV-1           3408
SIV             2119
HCV             1559
MCMV             923
YFV              542
Name: count, dtype: int64
**************************************************
There are 42 antigen species


### Calculate TCR distances to Database.

- metric=’identity’: For now, we only consider exact sequence matches. We will later introduce queries for similar sequences as well.
- sequence=’aa’: TCR sequences are compared on an amino-acid level instead of nucleobases, since specificity depends on the protein structure and this information is provided by the VDJdb.

In [6]:
metric = "identity"
sequence = "aa"
ir.pp.ir_dist(tcell_mdata, vdjdb, metric=metric, sequence=sequence)

### Query Database

In [7]:
sequence='aa'
metric='identity'
receptor_arm='VDJ'
ir.tl.ir_query(
    tcell_mdata["airr"],
    vdjdb,
    metric=metric,
    sequence=sequence,
    receptor_arms=receptor_arm,
    dual_ir="primary_only",
)

  0%|          | 0/1457 [00:00<?, ?it/s]

# Calculate TCR distances

To detect cells with shared specficity, we calculate the pairwise sequence distance between their TCRs.
- We are doing this to group cells within a dataset to increase the number of hits we get when we query a TCR database.

There are 3 ways to calculate the distances:
1. **Edit Distances:** This method calculates the minimum number of operations required to transform one sequence into another. These operations typically include insertions, deletions, and substitutions of single elements in the sequence. For example, the edit distance between "cat" and "hat" is 1 because only one substitution is needed (replace 'c' with 'h'). This method is useful in understanding how similar two sequences are and in identifying the potential evolutionary changes between them.

2. **k-mer Matching:** A k-mer is a substring of length 'k' from a longer sequence. The k-mer matching technique involves comparing the occurrence and frequency of these short motifs between two sequences. This method is often used for sequence alignment, genome assembly, and identifying sequence motifs that may have biological significance. For example, a 3-mer analysis of the sequence "ATGCTAGC" would involve looking at substrates like 'ATG', 'TGC', 'GCT', etc.

3. **Embeddings:** In bioinformatics, embeddings often refer to the process of representing sequences as vectors in a high-dimensional space, usually through deep learning models. Each sequence gets transformed into a numeric vector, capturing essential features and relationships within the data. This approach is beneficial for tasks like sequence classification, clustering, and prediction. It's akin to how words are embedded in natural language processing (NLP) tasks, capturing semantic relationships between words.

Focus on using either **TCRdist** or **TCRmatch**:

**TCRdist**
- uses the sequences of all CDRs and compares them via transformation cost and gap panelties [Dash et al., 2017].
- TCRdist requires complete information of CDR3 and V-gene of both chains.
 
**TCRmatch**
- uses all k-mers to compare the overlap in motifs between two TCRs based on their CDR3β sequences
    - Becuase it only requires CDR3β, we can use a wider array of data.

## TCRmatch

In [8]:
# Filter cells with missing beta chains
filter = ["no IR", "multichain", "ambiguous", "orphan VJ"]
beta_tcr = mdata[mdata["airr"].obs["chain_pairing"].isin(filter)]

# Keep only T cells
not_tcells = ["B intermediate", "Plasmablast"]
beta_tcr = beta_tcr[~beta_tcr["gex"].obs["consensus_celltype"].isin(not_tcells)]

In [9]:
beta_tcr["airr"].obs.columns

Index(['Sample_Tag', 'receptor_type', 'receptor_subtype', 'chain_pairing',
       'clone_id', 'clone_id_size', 'cc_aa_alignment', 'cc_aa_alignment_size',
       'clone_id:full', 'clone_id_size:full', 'cc_aa_alignment:full',
       'cc_aa_alignment_size:full', 'ID', 'clonal_expansion', 'is_full_chain'],
      dtype='object')

In [10]:
with ir.get.airr_context(beta_tcr, airr_variable="v_call", chain=("VDJ_1", "VJ_1")) as m:
    display(m.obs)

/Users/djlisko/gitrepos/rhapsody-analysis-notebook/.venv/lib/python3.11/site-packages/anndata/_core/aligned_mapping.py:64: ExperimentalFeatureWarning: Support for Awkward Arrays is currently experimental. Behavior may change in the future. Please report any issues you may encounter!
  warnings.warn(
/Users/djlisko/gitrepos/rhapsody-analysis-notebook/.venv/lib/python3.11/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,adt:Cell_Type_Experimental,adt:Sample_Tag,adt:Sample_Name,adt:SampleTag01_hs_Read_Count,adt:SampleTag02_hs_Read_Count,adt:SampleTag03_hs_Read_Count,adt:SampleTag04_hs_Read_Count,adt:SampleTag05_hs_Read_Count,adt:SampleTag06_hs_Read_Count,adt:SampleTag07_hs_Read_Count,...,airr:clone_id:full,airr:clone_id_size:full,airr:cc_aa_alignment:full,airr:cc_aa_alignment_size:full,airr:ID,airr:clonal_expansion,airr:is_full_chain,tcell_leiden,VDJ_1_v_call,VJ_1_v_call
1218,T_CD4_memory,SampleTag09_hs,SampleTag09_hs,5,15,7,18,2,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,False,4,NaN,TRAV5*01
6189,T_CD4_memory,SampleTag12_hs,SampleTag12_hs,17,64,8,6,23,44,205,...,NaN,NaN,NaN,NaN,NaN,NaN,False,13,TRBV2*01,IGKV4-1*01
6572,T_CD8_memory,SampleTag03_hs,SampleTag03_hs,36,14,5262,103,1,29,48,...,NaN,NaN,NaN,NaN,NaN,NaN,False,2,TRBV12-3*01,TRGV2*01
9608,T_CD4_memory,SampleTag09_hs,SampleTag09_hs,8,16,118,42,1,41,35,...,NaN,NaN,NaN,NaN,NaN,NaN,False,22,NaN,NaN
21537,T_CD8_naive,SampleTag12_hs,SampleTag12_hs,13,23,8,62,31,84,84,...,NaN,NaN,NaN,NaN,NaN,NaN,False,10,NaN,TRAV29/DV5*01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14033755,T_CD8_memory,SampleTag03_hs,SampleTag03_hs,21,133,12700,105,58,24,17,...,NaN,NaN,NaN,NaN,NaN,NaN,False,2,TRBV7-6*01,TRAV8-2*01
14039104,T_CD4_memory,SampleTag03_hs,SampleTag03_hs,12,3,8381,16,16,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,False,14,NaN,TRAV16*01
14041429,T_CD8_memory,SampleTag09_hs,SampleTag09_hs,30,51,19,65,8,47,69,...,NaN,NaN,NaN,NaN,NaN,NaN,False,9,NaN,NaN
14042115,T_CD4_naive,SampleTag09_hs,SampleTag09_hs,7,6,0,17,0,9,14,...,NaN,NaN,NaN,NaN,NaN,NaN,False,5,NaN,NaN


In [17]:
tcr_region = "VDJ_1_v_call"
cols = ["VDJ_1_junction_aa", "VDJ_1_v_call",  "VDJ_1_j_call"]

# for col in ["VJ_1_v_call", "VDJ_1_v_call"]:

with ir.get.airr_context(beta_tcr, ["junction_aa", "v_call", "j_call"]) as m:
    m = m[~m.obs[tcr_region].isna()]
    m.update()
   
    df_tcrdist = m.obs[
        [
            "VDJ_1_junction_aa",
            "VDJ_1_v_call",
            "VDJ_1_j_call",
            "airr:chain_pairing",
            "gex:ID",
            "airr:antigen.species",
            "gex:consensus_celltype",
        ]
    ].copy()

/Users/djlisko/gitrepos/rhapsody-analysis-notebook/.venv/lib/python3.11/site-packages/mudata/_core/mudata.py:760: ImplicitModificationWarning: Setting element `.varm['adt']` of view, initializing view as actual.
  attrm[mod] = mapping > 0
/Users/djlisko/gitrepos/rhapsody-analysis-notebook/.venv/lib/python3.11/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/djlisko/gitrepos/rhapsody-analysis-notebook/.venv/lib/python3.11/site-packages/mudata/_core/mudata.py:760: ImplicitModificationWarning: Setting element `.varm['gex']` of view, initializing view as actual.
  attrm[mod] = mapping > 0
/Users/djlisko/gitrepos/rhapsody-analysis-notebook/.venv/lib/python3.11/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/djlisko/gitrepos/rhap

KeyError: "['airr:antigen.species'] not in index"

In [19]:
beta_tcr["airr"].obs

,Sample_Tag,receptor_type,receptor_subtype,chain_pairing,clone_id,clone_id_size,cc_aa_alignment,cc_aa_alignment_size,clone_id:full,clone_id_size:full,cc_aa_alignment:full,cc_aa_alignment_size:full,ID,clonal_expansion,is_full_chain
cell_id,,,,,,,,,,,,,,,
1218,SampleTag09_hs,TCR,TRA+TRB,orphan VJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
6189,SampleTag12_hs,ambiguous,ambiguous,ambiguous,7,1.0,7,1.0,NaN,NaN,NaN,NaN,NaN,NaN,False
6572,SampleTag03_hs,TCR,ambiguous,ambiguous,8,3.0,8,3.0,NaN,NaN,NaN,NaN,NaN,NaN,False
9608,SampleTag09_hs,no IR,no IR,no IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
21537,SampleTag12_hs,TCR,TRA+TRB,orphan VJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14033755,SampleTag03_hs,TCR,ambiguous,ambiguous,1583,1.0,516,2.0,NaN,NaN,NaN,NaN,NaN,NaN,False
14039104,SampleTag03_hs,TCR,TRA+TRB,orphan VJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
14041429,SampleTag09_hs,no IR,no IR,no IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [20]:
tcell_mdata

MuData object with n_obs × n_vars = 2822 × 4021
  obs:	'tcell_leiden'
  var:	'Raw_Reads', 'Raw_Molecules', 'Raw_Seq_Depth', 'RSEC_Adjusted_Molecules', 'RSEC_Adjusted_Reads_non-singleton', 'RSEC_Adjusted_Molecules_non-singleton', 'Gene_Expression', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
  3 modalities
    adt:	2822 x 21
      obs:	'Cell_Type_Experimental', 'Sample_Tag', 'Sample_Name', 'SampleTag01_hs_Read_Count', 'SampleTag02_hs_Read_Count', 'SampleTag03_hs_Read_Count', 'SampleTag04_hs_Read_Count', 'SampleTag05_hs_Read_Count', 'SampleTag06_hs_Read_Count', 'SampleTag07_hs_Read_Count', 'SampleTag08_hs_Read_Count', 'SampleTag09_hs_Read_Count', 'SampleTag10_hs_Read_Count', 'SampleTag11_hs_Read_Count', 'SampleTag12_hs_Read_Count', 'Total_VDJ_Read_Count', 'Total_VDJ_Molecule_Count', 'BCR_Heavy_V_gene_Dominant', 'BCR_Heavy_D_gene_Dominant', 'BCR_Heavy_J_gene_Dominant', 'BCR_Heavy_C_gene_Dominant', 'BCR_Heavy_CDR3_Nucleotide_Dominant', 'BCR_Heavy_CDR3_Translation_Dominant', 'BCR_Heavy_Read_Count', 'BCR_Heavy_Molecule_Count', 'BCR_Light_V_gene_Dominant', 'BCR_Light_J_gene_Dominant', 'BCR_Light_C_gene_Dominant', 'BCR_Light_CDR3_Nucleotide_Dominant', 'BCR_Light_CDR3_Translation_Dominant', 'BCR_Light_Read_Count', 'BCR_Light_Molecule_Count', 'TCR_Alpha_Gamma_V_gene_Dominant', 'TCR_Alpha_Gamma_J_gene_Dominant', 'TCR_Alpha_Gamma_C_gene_Dominant', 'TCR_Alpha_Gamma_CDR3_Nucleotide_Dominant', 'TCR_Alpha_Gamma_CDR3_Translation_Dominant', 'TCR_Alpha_Gamma_Read_Count', 'TCR_Alpha_Gamma_Molecule_Count', 'TCR_Beta_Delta_V_gene_Dominant', 'TCR_Beta_Delta_D_gene_Dominant', 'TCR_Beta_Delta_J_gene_Dominant', 'TCR_Beta_Delta_C_gene_Dominant', 'TCR_Beta_Delta_CDR3_Nucleotide_Dominant', 'TCR_Beta_Delta_CDR3_Translation_Dominant', 'TCR_Beta_Delta_Read_Count', 'TCR_Beta_Delta_Molecule_Count', 'BCR_Paired_Chains', 'TCR_Paired_Chains', 'ID', 'samle_number', 'day', 'date', 'time_points', 'sample_type', 'aghvd', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'n_counts'
      var:	'Raw_Reads', 'Raw_Molecules', 'Raw_Seq_Depth', 'RSEC_Adjusted_Molecules', 'RSEC_Adjusted_Reads_non-singleton', 'RSEC_Adjusted_Molecules_non-singleton', 'Gene_Expression', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'adt_names'
      uns:	'Pipeline_Inputs', 'Pipeline_Metrics'
      obsm:	'X_tsne'
      layers:	'counts'
    gex:	2822 x 4000
      obs:	'Cell_Type_Experimental', 'Sample_Tag', 'Sample_Name', 'SampleTag01_hs_Read_Count', 'SampleTag02_hs_Read_Count', 'SampleTag03_hs_Read_Count', 'SampleTag04_hs_Read_Count', 'SampleTag05_hs_Read_Count', 'SampleTag06_hs_Read_Count', 'SampleTag07_hs_Read_Count', 'SampleTag08_hs_Read_Count', 'SampleTag09_hs_Read_Count', 'SampleTag10_hs_Read_Count', 'SampleTag11_hs_Read_Count', 'SampleTag12_hs_Read_Count', 'Total_VDJ_Read_Count', 'Total_VDJ_Molecule_Count', 'BCR_Heavy_V_gene_Dominant', 'BCR_Heavy_D_gene_Dominant', 'BCR_Heavy_J_gene_Dominant', 'BCR_Heavy_C_gene_Dominant', 'BCR_Heavy_CDR3_Nucleotide_Dominant', 'BCR_Heavy_CDR3_Translation_Dominant', 'BCR_Heavy_Read_Count', 'BCR_Heavy_Molecule_Count', 'BCR_Light_V_gene_Dominant', 'BCR_Light_J_gene_Dominant', 'BCR_Light_C_gene_Dominant', 'BCR_Light_CDR3_Nucleotide_Dominant', 'BCR_Light_CDR3_Translation_Dominant', 'BCR_Light_Read_Count', 'BCR_Light_Molecule_Count', 'TCR_Alpha_Gamma_V_gene_Dominant', 'TCR_Alpha_Gamma_J_gene_Dominant', 'TCR_Alpha_Gamma_C_gene_Dominant', 'TCR_Alpha_Gamma_CDR3_Nucleotide_Dominant', 'TCR_Alpha_Gamma_CDR3_Translation_Dominant', 'TCR_Alpha_Gamma_Read_Count', 'TCR_Alpha_Gamma_Molecule_Count', 'TCR_Beta_Delta_V_gene_Dominant', 'TCR_Beta_Delta_D_gene_Dominant', 'TCR_Beta_Delta_J_gene_Dominant', 'TCR_Beta_Delta_C_gene_Dominant', 'TCR_Beta_Delta_CDR3_Nucleotide_Dominant', 'TCR_Beta_Delta_CDR3_Translation_Dominant', 'TCR_Beta_Delta_Read_Count', 'TCR_Beta_Delta_Molecule_Count', 'BCR_Paired_Chai